Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

- All approaches in notebook 01 load all the data into memory. A very large file might fill up memory. 
- Counting words in each line is totally independent of the others. 
- We can evaluate each piece of data and immediately free up the memory space. 
- Data chunks would be small enough not to stress memory, but big enough for efficient use of the CPU.

In this notebook we will see how to divide the load between different processes.

# Container datatypes

`collection` module implements specialized container datatypes providing alternatives to Python’s general purpose built-in containers, `dict`, `list`, `set`, and `tuple`.

- `namedtuple()`	: factory function for creating tuple subclasses with named fields
- `deque`	: list-like container with fast appends and pops on either end
- `ChainMap`	: dict-like class for creating a single view of multiple mappings
- `Counter`	: dict subclass for counting hashable objects
- `defaultdict` :	dict subclass that calls a factory function to supply missing values


## Counter

A Counter is a dict subclass for counting hashable objects. It is an unordered collection where elements are stored as dictionary keys and their counts are stored as dictionary values. Counts are allowed to be any integer value including zero or negative counts. The Counter class is similar to bags or multisets in other languages.

Elements are counted from an iterable or initialized from another mapping (or counter):

In [225]:
from collections import Counter

violet = dict(r=23,g=13,b=23)
cnt = Counter(violet)  # or Counter(r=238, g=130, b=238)
print(cnt['c'])
print(cnt['r'])

0
23


In [226]:
print(*cnt.elements())

r r r r r r r r r r r r r r r r r r r r r r r g g g g g g g g g g g g g b b b b b b b b b b b b b b b b b b b b b b b


In [199]:
cnt.most_common(2)

[('r', 238), ('b', 238)]

In [201]:
cnt.values()

dict_values([238, 130, 238])

### Exercise 2.1

Use a `Counter` object to count words occurences in `text` produced by the `lorem` module.

The Counter class is similar to bags or multisets in some Python libraries or other languages. We will see later how to use Counter-like objects in a parallel context. 

## Partition data

In order to parallelize **reduce** operation, 
data must be aligned in a container. For this operation we will use the
`dict` subclass `defaultdict`.



## defaultdict

`dict` subclass that calls a factory function to supply missing values.
Using list as the default_factory, it is easy to group a sequence of key-value pairs into a dictionary of lists:





In [230]:
from collections import defaultdict
s = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
d = defaultdict(list)
for k, v in s:
    d[k].append(v)

sorted(d.items())

[('blue', [2, 4]), ('red', [1]), ('yellow', [1, 3])]

### Exercise 2.2

By setting the default_factory to `int`, use the defaultdict for counting words in a text created by lorem module:




### Exercise 2.3

Create a function named `partition` that stores the key/value pairs from `words` (function created in notebook 1) into a `defaultdict` from `collections` module. Output will be:
```python
[('word1', [1, 1]), ('word2', [1]), ('word3', [1, 1, 1])]
```

- [itertools.chain(*mapped_values)](https://docs.python.org/3.6/library/itertools.html#itertools.chain) is used for treating consecutive sequences as a single sequence. 
- [operator](https://docs.python.org/3/library/operator.html).itemgetter(1)
Return a callable object that fetches item from its operand using the operand’s __getitem__() method. 
```python
inventory = [('apple', 3), ('banana', 2), ('pear', 5), ('orange', 1)]
getcount = itemgetter(1)
>>> list(map(getcount, inventory))
[3, 2, 5, 1]
>>> sorted(inventory, key=getcount)
[('orange', 1), ('banana', 2), ('apple', 3), ('pear', 5)]
```

In [233]:
import multiprocessing as mp
def words_mp(file):
    """
    Read a text file and return a sorted list of (word, 1) values.
    """
    print(mp.current_process().name, 'reading', file)
    translator = str.maketrans('', '', string.punctuation)
    output = []
    try:
        with open(file) as f:
            for line in f:   
                line = line.strip()
                line = line.translate(translator)
                for word in line.split():
                    if word.isalpha():
                        word = word.lower()
                        output.append((word, 1))
                        
    except UnicodeDecodeError as err:
        print("Some error occurred decoding file %s: %s" % (file, err))
                
    output.sort()
    return output

In [234]:
import collections
def partition_mp(mapped_values):
    """
        Organize the mapped values by their key.
        Returns an unsorted sequence of tuples with
        a key and a sequence of values.
    """
    partitioned_data = collections.defaultdict(list)
    for key, value in mapped_values:
        partitioned_data[key].append(value)
    return partitioned_data.items()

In [ ]:
def reduce_mp(item):
    """Convert the partitioned data for a word to a
    tuple containing the word and the number of occurances.
    """
    word, occurances = item
    return (word, sum(occurances))

In [ ]:
import glob, itertools, operator
files = glob.glob('*.txt')

mapped_values = map(words,files)
partitioned_data = partition_mp(itertools.chain(*mapped_values))
reduced_values = map(reduce, partitioned_data)
reduced_values.sort(key=operator.itemgetter(1)) # sort values by number of occurences
reduced_values.reverse() # Put highest number of occurences on top